## This notebook is part of Hadoop and Spark training delivered by IT-DB group
### SPARK Streaming Hands-On Lab
_ by Prasanth Kothuri _

### Hands-On 1 - Stream processing using Spark Streaming and Kafka
*This demonstrates processing of unbounded data from Kafka topic and perform simple string manipulations and aggregations*

#### Import the required modules

In [1]:
import os
import json
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

#### Make spark streaming kafka module available to Spark executors

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0 pyspark-shell'

#### Create SparkContext

In [3]:
conf = SparkConf().setMaster("local[*]").set("spark.driver.memory", "4g")
sc = SparkContext(conf = conf)

#### Create streaming context

In [4]:
ssc = StreamingContext(sc, 60)

#### Hook upto kafka topic

In [5]:
kafkaStream = KafkaUtils.createStream(ssc, 'sstreaming:2181', 'spark-streaming-pkothuri', {'twitter_json':1})

#### Parse the messages into json

In [6]:
tweets_json = kafkaStream.map(lambda x: json.loads(x[1]))

#### Number of tweets in each batch

In [7]:
tweets_json.count().map(lambda x:'Number of tweets in this batch: %s' % x).pprint()

#### json returned from kafka
```json
{
  "payload": {
    "id": 935610640634449900,
    "created_at": "2017-11-28T20:45:34.000+0000",
    "user": {
      "id": 290028823,
      "name": "TDB68",
      "screen_name": "tbell1968",
      "location": "Dallas",
      "verified": false,
      "friends_count": 3031,
      "followers_count": 2755,
      "statuses_count": 60606
    },
    "text": "RT @paulkrugman: At this point everyone should know that dynamic scoring won't make much difference; JCT estimate likely to look a l… ",
    "lang": "en",
    "is_retweet": true,
    "entities": {
      "hashtags": [],
      "media": [],
      "urls": [],
      "user_mentions": [
        {
          "id": 17006157,
          "name": "Paul Krugman",
          "screen_name": "paulkrugman"
        }
      ]
    }
  }
}
```

#### Count tweets by location

In [8]:
location_counts = tweets_json.map(lambda tweet: tweet['payload']['user']['location']).countByValue()

In [9]:
top_locations = location_counts \
    .transform( (lambda foo:foo .sortBy(lambda x:( -x[1]))) ) \
    .transform(lambda rdd:sc.parallelize(rdd.take(5)))

In [10]:
top_locations.pprint()

#### High frequency words in the tweets

In [11]:
tweets_json \
    .flatMap(lambda tweet:tweet['payload']['text'].split(" ")) \
    .countByValue() \
    .transform(lambda rdd:rdd.sortBy(lambda x:-x[1])) \
    .pprint()

#### Start the streaming context

In [12]:
ssc.start()
ssc.awaitTermination(timeout=180)

-------------------------------------------
Time: 2018-04-17 09:31:00
-------------------------------------------
Number of tweets in this batch: 42

-------------------------------------------
Time: 2018-04-17 09:31:00
-------------------------------------------
(None, 7)
(u'Arnhem, Nederland', 3)
(u'Haute Normandie et Vend\xe9e', 2)
(u'under the sky of England', 2)
(u'London | Brussels | Geneva ', 1)

-------------------------------------------
Time: 2018-04-17 09:31:00
-------------------------------------------
(u'RT', 35)
(u'the', 33)
(u'of', 18)
(u'in', 16)
(u'to', 11)
(u'is', 7)
(u'and', 6)
(u'on', 6)
(u'#Syria', 6)
(u'for', 5)
...

-------------------------------------------
Time: 2018-04-17 09:32:00
-------------------------------------------
Number of tweets in this batch: 78

-------------------------------------------
Time: 2018-04-17 09:32:00
-------------------------------------------
(None, 21)
(u'Toronto, Canada', 4)
(u'Londoner', 2)
(u'Italia', 2)
(u'Ireland', 2)

----

#### stop the streaming context

In [13]:
ssc.stop()

### Hands-On 2 - Stream processing using Spark Streaming and Kafka
*This demonstrates spark streaming window operations*

#### Restart the kernel to clear all the variables; we are going to create streaming context again
In the top menu, Kernel -> Restart

#### Import the required modules

In [1]:
import os
import json
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

#### Make spark streaming kafka module available to Spark executors

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0 pyspark-shell'

#### Window functionality stream processing code

In [3]:
def createStreamingContext():  
    conf = SparkConf().setMaster("local[*]").set("spark.driver.memory", "2g").set("spark.executor.memory", "2g")
    sc = SparkContext(conf = conf)
    ssc = StreamingContext(sc, 10)

    # Hook upto kafka topic (streamingcontext, zookeeper endpoint, consumer_name, dictionary of topic and offset)
    kafkaStream = KafkaUtils.createStream(ssc, 'sstreaming:2181', 'spark-streaming-pkothuri', {'twitter_json':1})

    # Extract and parse the tweets
    tweets_json = kafkaStream.map(lambda x: json.loads(x[1]))
    
    # Number of tweets in this batch
    count_this_batch = kafkaStream.count().map(lambda x:('Tweets this batch: %s' % x))

    # One minute rolling counts
    count_windowed = kafkaStream.countByWindow(60,10).map(lambda x:('Tweets total (One minute rolling count): %s' % x))
    
    # Top 10 hashtags
    hashTags = tweets_json.map(lambda tweet: tweet['payload']['text']) \
                            .flatMap(lambda x : x.split(" ")) \
                            .filter(lambda x: x.startswith("#"))
                                                  
    count_hashTags = hashTags.countByValue() \
                                .transform(lambda rdd:rdd .sortBy(lambda x:-x[1])) \
                                .map(lambda x:"hashTag counts this batch:\tValue %s\tCount %s" % (x[0],x[1]))
                                                  
    count_hashTags_windowed = hashTags.countByValue() \
                                .transform(lambda rdd:rdd .sortBy(lambda x:-x[1])) \
                                .map(lambda x:"hashTag counts one minute rolling:\tValue %s\tCount %s" % (x[0],x[1]))

    # Write totals to stdout
    count_this_batch.pprint()
    count_windowed.pprint()                              
    count_hashTags.pprint(5)
    count_hashTags_windowed.pprint(5)

    return ssc

#### Start the streaming context

In [4]:
# checkpoint directory required for windowing functionality
ssc = StreamingContext.getOrCreate('/tmp/checkpoint001',lambda: createStreamingContext())  
ssc.start()  
ssc.awaitTermination() 

-------------------------------------------
Time: 2018-04-17 09:40:10
-------------------------------------------
Tweets this batch: 593

-------------------------------------------
Time: 2018-04-17 09:40:10
-------------------------------------------
Tweets total (One minute rolling count): 593

-------------------------------------------
Time: 2018-04-17 09:40:10
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 138
hashTag counts this batch:	Value #Israel	Count 23
hashTag counts this batch:	Value #Douma	Count 23
hashTag counts this batch:	Value #Russia	Count 21
hashTag counts this batch:	Value #Syria:	Count 17
...

-------------------------------------------
Time: 2018-04-17 09:40:10
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 138
hashTag counts one minute rolling:	Value #Israel	Count 23
hashTag counts one minute rolling:	Value #Douma	Count 23
hashTag counts one minute rolling:	Value #Russi

-------------------------------------------
Time: 2018-04-17 09:41:30
-------------------------------------------
Tweets total (One minute rolling count): 72

-------------------------------------------
Time: 2018-04-17 09:41:30
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 1
hashTag counts this batch:	Value #ToriesOut
The	Count 1
hashTag counts this batch:	Value #Syria,	Count 1
hashTag counts this batch:	Value #Syria.	Count 1
hashTag counts this batch:	Value #FalseFlag	Count 1
...

-------------------------------------------
Time: 2018-04-17 09:41:30
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 1
hashTag counts one minute rolling:	Value #ToriesOut
The	Count 1
hashTag counts one minute rolling:	Value #Syria,	Count 1
hashTag counts one minute rolling:	Value #Syria.	Count 1
hashTag counts one minute rolling:	Value #FalseFlag	Count 1
...

-------------------------------------------
Time: 2018-

-------------------------------------------
Time: 2018-04-17 09:42:50
-------------------------------------------
Tweets this batch: 16

-------------------------------------------
Time: 2018-04-17 09:42:50
-------------------------------------------
Tweets total (One minute rolling count): 82

-------------------------------------------
Time: 2018-04-17 09:42:50
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 4
hashTag counts this batch:	Value #Russia	Count 2
hashTag counts this batch:	Value #SyrianArmy	Count 1
hashTag counts this batch:	Value #West's	Count 1
hashTag counts this batch:	Value #Syria,"	Count 1
...

-------------------------------------------
Time: 2018-04-17 09:42:50
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 4
hashTag counts one minute rolling:	Value #Russia	Count 2
hashTag counts one minute rolling:	Value #SyrianArmy	Count 1
hashTag counts one minute rolling:	Value #West's

-------------------------------------------
Time: 2018-04-17 09:44:10
-------------------------------------------
Tweets this batch: 15

-------------------------------------------
Time: 2018-04-17 09:44:10
-------------------------------------------
Tweets total (One minute rolling count): 84

-------------------------------------------
Time: 2018-04-17 09:44:10
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 3
hashTag counts this batch:	Value #Israel	Count 2
hashTag counts this batch:	Value #Syria;	Count 2
hashTag counts this batch:	Value #LBC	Count 1
hashTag counts this batch:	Value #infosec…	Count 1
...

-------------------------------------------
Time: 2018-04-17 09:44:10
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 3
hashTag counts one minute rolling:	Value #Israel	Count 2
hashTag counts one minute rolling:	Value #Syria;	Count 2
hashTag counts one minute rolling:	Value #LBC	Count 1
hash

-------------------------------------------
Time: 2018-04-17 09:45:30
-------------------------------------------
Tweets this batch: 12

-------------------------------------------
Time: 2018-04-17 09:45:30
-------------------------------------------
Tweets total (One minute rolling count): 79

-------------------------------------------
Time: 2018-04-17 09:45:30
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 3
hashTag counts this batch:	Value #WorldHeritage	Count 1
hashTag counts this batch:	Value #Ольгино	Count 1
hashTag counts this batch:	Value #SYRIA	Count 1
hashTag counts this batch:	Value #France	Count 1
...

-------------------------------------------
Time: 2018-04-17 09:45:30
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 3
hashTag counts one minute rolling:	Value #WorldHeritage	Count 1
hashTag counts one minute rolling:	Value #Ольгино	Count 1
hashTag counts one minute rolling:	Value #

...

-------------------------------------------
Time: 2018-04-17 10:30:30
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 5
hashTag counts one minute rolling:	Value #Israel	Count 3
hashTag counts one minute rolling:	Value #Russia-ns…	Count 2
hashTag counts one minute rolling:	Value #Syria-n	Count 1
hashTag counts one minute rolling:	Value #US	Count 1
...

-------------------------------------------
Time: 2018-04-17 10:30:40
-------------------------------------------
Tweets this batch: 14

-------------------------------------------
Time: 2018-04-17 10:30:40
-------------------------------------------
Tweets total (One minute rolling count): 81

-------------------------------------------
Time: 2018-04-17 10:30:40
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 3
hashTag counts this batch:	Value #Israel	Count 2
hashTag counts this batch:	Value #Trump,	Count 1
hashTag counts this batch:	Value #Kashmi

-------------------------------------------
Time: 2018-04-17 10:31:50
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 5
hashTag counts this batch:	Value #syria-n	Count 1
hashTag counts this batch:	Value #EastGhouta	Count 1
hashTag counts this batch:	Value #US	Count 1
hashTag counts this batch:	Value #Israel	Count 1
...

-------------------------------------------
Time: 2018-04-17 10:31:50
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 5
hashTag counts one minute rolling:	Value #syria-n	Count 1
hashTag counts one minute rolling:	Value #EastGhouta	Count 1
hashTag counts one minute rolling:	Value #US	Count 1
hashTag counts one minute rolling:	Value #Israel	Count 1
...

-------------------------------------------
Time: 2018-04-17 10:32:00
-------------------------------------------
Tweets this batch: 15

-------------------------------------------
Time: 2018-04-17 10:32:00
-------------------------

-------------------------------------------
Time: 2018-04-17 10:33:10
-------------------------------------------
Tweets this batch: 15

-------------------------------------------
Time: 2018-04-17 10:33:10
-------------------------------------------
Tweets total (One minute rolling count): 80

-------------------------------------------
Time: 2018-04-17 10:33:10
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 3
hashTag counts this batch:	Value #Syria's	Count 1
hashTag counts this batch:	Value #Duma:	Count 1
hashTag counts this batch:	Value #Russia	Count 1
hashTag counts this batch:	Value #chemicalatack.	Count 1
...

-------------------------------------------
Time: 2018-04-17 10:33:10
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 3
hashTag counts one minute rolling:	Value #Syria's	Count 1
hashTag counts one minute rolling:	Value #Duma:	Count 1
hashTag counts one minute rolling:	Value #Russia	

-------------------------------------------
Time: 2018-04-17 10:34:20
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria's	Count 1
hashTag counts one minute rolling:	Value #Syria	Count 1
hashTag counts one minute rolling:	Value #Tories	Count 1
hashTag counts one minute rolling:	Value #Israel	Count 1
hashTag counts one minute rolling:	Value #Syria
#बेटी_मांगे_इंसाफ	Count 1
...

-------------------------------------------
Time: 2018-04-17 10:34:30
-------------------------------------------
Tweets this batch: 19

-------------------------------------------
Time: 2018-04-17 10:34:30
-------------------------------------------
Tweets total (One minute rolling count): 78

-------------------------------------------
Time: 2018-04-17 10:34:30
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 6
hashTag counts this batch:	Value #Syria.	Count 2
hashTag counts this batch:	Value #syria-n	Count 1
hashTag counts this ba

-------------------------------------------
Time: 2018-04-17 10:35:40
-------------------------------------------
hashTag counts this batch:	Value #Syria	Count 2
hashTag counts this batch:	Value #Israel	Count 2
hashTag counts this batch:	Value #UK	Count 2
hashTag counts this batch:	Value #StopBiafraKillings	Count 1
hashTag counts this batch:	Value #Isr…	Count 1
...

-------------------------------------------
Time: 2018-04-17 10:35:40
-------------------------------------------
hashTag counts one minute rolling:	Value #Syria	Count 2
hashTag counts one minute rolling:	Value #Israel	Count 2
hashTag counts one minute rolling:	Value #UK	Count 2
hashTag counts one minute rolling:	Value #StopBiafraKillings	Count 1
hashTag counts one minute rolling:	Value #Isr…	Count 1
...

-------------------------------------------
Time: 2018-04-17 10:35:50
-------------------------------------------
Tweets this batch: 16

-------------------------------------------
Time: 2018-04-17 10:35:50
---------------

In [ ]:
ssc.stop()